In [4]:
# A class to store a Trie node
class Trie:
    # Constructor
    def __init__(self):
        self.isLeaf = False
        self.children = {}
 
 
    # Iterative function to insert a string into a Trie
    def insert(self, key):
        #print('Inserting…', key)
 
        # start from the root node
        curr = self
 
        # do for each character of the key
        for c in key:
            # go to the next node and create one if the path doesn't exist
            curr = curr.children.setdefault(c, Trie())
 
        # mark the current node as a leaf
        curr.isLeaf = True
 
 
    # Iterative function to search a key in a Trie. It returns True
    # if the key is found in the Trie; otherwise, it returns False
    def search(self, key):
 
        #print('Searching', key, end=': ')
        curr = self
 
        # do for each character of the key
        for c in key:
            # go to the next node
            curr = curr.children.get(c)
            # if the string is invalid (reached end of a path in the Trie)
            if curr is None:
                return False
 
        # return true if the current node is a leaf node, and we have reached
        # the end of the string
        return curr.isLeaf
    
    
    
# Creating an Instance of the class trie
t = Trie()

In [5]:
#Open the training corpus

file= open("train.ig.txt", encoding="utf8")
file_read= file.read()
file_to_count= file_read.split()
my_file= list(set(file_to_count))
file_count= len(my_file)
print(file_count)
print(file_to_count[:10])
file.close()

my_list_of_words= my_file
#my_list_of_words=["ọ", "dị", "paul", "mkpa", "ịgbanwe", "bọlb", "ọkụ", "eletrik", "dị", "nihu", "ụlọ", "ya"]

#Ọ bụ n’ihi na ọ ga eji ubube rụọ ha
#ọ dị paul mkpa ịgbanwe bọlb ọkụ eletrik dị nihu ụlọ ya
#Iji Ubube Eme Ihe — Ị̀ Na - eme Nnyocha Ndị A Iji Zere Mmerụ Ahụ ?

28538
['mgbe', 'jezibel', 'nwụsịrị', 'eze', 'jihu', 'egbughị', 'oge', 'ọ', 'bụla', 'nigbusịsị']


In [6]:
for word in my_list_of_words:
    t.insert(word)
    
t.search("mmadụ")

True

In [7]:
# Run the models

#Using Model files to call ngram. (This causes my computer to hang)
from collections import Counter, defaultdict
import pickle
def _my_normalization():
    return defaultdict(int)
    
    
    
with open("Tri_Left.pkl", "rb") as pkl_handle:
	model_Tri_Left = pickle.load(pkl_handle)
    
with open("Tri_Right.pkl", "rb") as pkl_handle:
	model_Tri_Right = pickle.load(pkl_handle)
    
with open("Tri_Center.pkl", "rb") as pkl_handle:
	model_Tri_Center = pickle.load(pkl_handle)
    
with open("Bi_Left.pkl", "rb") as pkl_handle:
	model_Bi_Left = pickle.load(pkl_handle)
    
with open("Bi_Right.pkl", "rb") as pkl_handle:
	model_Bi_Right = pickle.load(pkl_handle)
    


## Ngram Model Candidate Suggestions

In [8]:
# NGRAM RESULTS

import re
import collections


def func_of_ifs(dici_list):
    index_mod=[]
    counter= collections.Counter()
    
    for mod in dici_list:#then get the indexes of words that are correct
        if mod:
            #print(mod)
            index_mod.append(dici_list.index(mod))
    for dict_el in dici_list:
        if dict_el:
            counter.update(dict_el)
            
    return counter,index_mod


def ngram_multi_sugg(sent,arr_index):
    
    #sent_clean= re.sub(r'[^A-Za-z àáéèọíìịụụ́ụ̀úù]','',sent) #remove punctuation and numbers. I have to add Igbo characters to this??!
    sent_clean=re.sub(r"[^\w\s’]",'',sent)#Use regex to clean sentence
    arr_sent= sent_clean.split()# make sent an array
    index_mod=''
    #counter= collections.Counter()
    counter=''
    ngram_result= {}
    

  
    
    for word in arr_sent:
        # Check all the possible ngram variations of the word
       
        
        if word == arr_sent[arr_index]:
            
            #if len(arr_sent) < 2:
                #pass
            
            if len(arr_sent) == 2:
                
                if arr_index== 0: #first word
                    bi_left= dict(model_Bi_Left[arr_sent[arr_index+1]])
                    first_word= bi_left
                    ngram_result= first_word
                
                if arr_index==1: #second word
                    bi_right= dict(model_Bi_Right[arr_sent[arr_index-1]])
                    second_word= bi_right
                    ngram_result= second_word
                   
                
              
            if len(arr_sent) == 3:
                
                if arr_index== 0: #first word
                    tri_left= dict(model_Tri_Left[arr_sent[arr_index+1],arr_sent[arr_index+2]])
                    bi_left= dict(model_Bi_Left[arr_sent[arr_index+1]])
                    first_word= [tri_left,bi_left]
                    counter,index_mod= func_of_ifs(first_word)
            
                elif arr_index==1: #second word
                    bi_right= dict(model_Bi_Right[arr_sent[arr_index-1]])
                    bi_left= dict(model_Bi_Left[arr_sent[arr_index+1]])
                    second_word= [bi_right,bi_left]
                    counter,index_mod= func_of_ifs(second_word)
                    
                elif arr_index == 2: #third word
                    bi_right = dict(model_Bi_Right[arr_sent[arr_index-1]])
                    tri_right = dict(model_Tri_Right[arr_sent[arr_index-2],arr_sent[arr_index-1]])
                    third_word = [bi_right, tri_right]
                    counter,index_mod= func_of_ifs(third_word)
                                 
                counter= dict(counter)

                for el in counter:
                    ngram_result[el]=counter[el]/len(index_mod)
                  
                
                                  
            if len(arr_sent)>3:         
            
                if arr_index == 0: #first word
                    tri_left= dict(model_Tri_Left[arr_sent[arr_index+1],arr_sent[arr_index+2]])
                    bi_left= dict(model_Bi_Left[arr_sent[arr_index+1]])
                    first_word= [tri_left,bi_left]
                    counter,index_mod= func_of_ifs(first_word)


                elif arr_index == 1: #secondword
                    tri_center= dict(model_Tri_Center[arr_sent[arr_index-1],arr_sent[arr_index+1]])
                    bi_right= dict(model_Bi_Right[arr_sent[arr_index-1]])
                    bi_left= dict(model_Bi_Left[arr_sent[arr_index+1]])
                    tri_left= dict(model_Tri_Left[arr_sent[arr_index+1],arr_sent[arr_index+2]])
                    second_word= [tri_center,bi_right,bi_left,tri_left]
                    counter,index_mod= func_of_ifs(second_word)


                elif arr_index == len(arr_sent)-2: #second to the last
                    tri_center= dict(model_Tri_Center[arr_sent[arr_index-1],arr_sent[arr_index+1]])
                    bi_right= dict(model_Bi_Right[arr_sent[arr_index-1]])
                    bi_left= dict(model_Bi_Left[arr_sent[arr_index+1]])
                    tri_right= dict(model_Tri_Right[arr_sent[arr_index-2],arr_sent[arr_index-1]])
                    second_to_last_word= [tri_center,bi_left,bi_right,tri_right]
                    counter,index_mod= func_of_ifs(second_to_last_word)

                elif arr_index == len(arr_sent)-1: #last word
                    tri_right= dict(model_Tri_Right[arr_sent[arr_index-2],arr_sent[arr_index-1]])
                    bi_right= dict(model_Bi_Right[arr_sent[arr_index-1]])
                    last_word= [tri_right,bi_right]
                    counter,index_mod= func_of_ifs(last_word)

                else:
                    tri_right= dict(model_Tri_Right[arr_sent[arr_index-2],arr_sent[arr_index-1]])
                    tri_left= dict(model_Tri_Left[arr_sent[arr_index+1],arr_sent[arr_index+2]])
                    tri_center= dict(model_Tri_Center[arr_sent[arr_index-1],arr_sent[arr_index+1]])
                    bi_right= dict(model_Bi_Right[arr_sent[arr_index-1]])
                    bi_left= dict(model_Bi_Left[arr_sent[arr_index+1]])
                    all_others= [tri_right,tri_left,tri_center,bi_right,bi_left] #put them in a list
                    counter,index_mod= func_of_ifs(all_others)
                        
                    
                counter= dict(counter)

                for el in counter:
                    ngram_result[el]=counter[el]/len(index_mod)                
   
    return  ngram_result
            
        
    
    
    

    # PROBLEMS: Any of the models that runs out of words cannot be used as they will generate index errors. Hence, 
    # all the models cannot be used with all the words. The models used for the cumulative calculation should take into
    # the position of the words they are addressing just like in ngram_suggestion() above.
    
    #The regex is bad at identifying 
    
    # LINK: https://tutorial.eyehunts.com/python/python-dictionary-sum-values-with-the-same-key-example-code/

## Candidate Suggestions with Leveinsthein

In [9]:
import enchant

def candidate_suggestions (word,error_word):
    
    one_levensthein_words=[]
    
    words=list(word)
    top_words={}
    
    for el in words:
        if not el:
            words.pop(words.index(el))

    for key in words:
        #print(key) 
        #print(enchant.utils.levenshtein(key, error_word))    
        if enchant.utils.levenshtein(key, error_word)<=1 or enchant.utils.levenshtein(key, error_word)<=2:
            one_levensthein_words.append(key)
            
            
    for el in list(set(one_levensthein_words)):
        if el in words:
            top_words[el]= word[el]
           
    top_words= sorted(top_words.items(), key=lambda x: x[1], reverse=True)
    top_words= top_words[:5]
    top_words_and_probabilities = dict(top_words) # This gives the top words as well as the necessary probabilities
    top_words= list(dict(top_words)) 
    
    return top_words, top_words_and_probabilities
            



## Candidate Suggestions with Norvig

In [10]:
#def norvig_candidates(my_ngram,err_word):
def candidate_suggestions(my_ngram,err_word):

    
    import enchant
    import re
    from collections import Counter
    
    words= list(my_ngram)
    top_words={}
    
    def ngram_result(your_ngram):
        #open wordlist
        words_string=[]
        
        for key in your_ngram:
            words_string.append(key)
        return words_string
        #def words(text): return re.findall(r'\w+', text.lower())
        #def words(text): return re.findall(r'\w+', lower_my_list(words_from_ngram_list))

    #WORDS = Counter(words(open('igbo_clean_main.txt', encoding= 'utf8').read()))
    
    #words_from_ngram = file_to_count
    words_from_ngram_list = ngram_result(my_ngram)
    
    WORDS = Counter(words_from_ngram_list)
    
   

    def P(word, N=sum(WORDS.values())): 
        "Probability of `word`."
        return WORDS[word] / N

    def correction(word): 
        "Most probable spelling correction for word."
        return max(candidates(word), key=P)

    def candidates(word): 
        "Generate possible spelling corrections for word."
        """or known(edits3(word))"""#Add to the return statement below to run for 3 edit depth
        return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

    def known(words): 
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in WORDS)

    def edits1(word):
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghiịjklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)


    """Done the edit for up to 3, however this has substantially reduced the speed and time it takes
    to generate a correction, so I will make it dormant for now
    def edits3(word): 
        "All edits that are three edits away from `word`."
        return (e3 for e2 in edits2(word) for e3 in edits1(e2))"""

    def edits2(word): 
        "All edits that are two edits away from `word`."
        return (e2 for e1 in edits1(word) for e2 in edits1(e1))
    
    my_candidates= candidates(err_word)
    
    for el in list(my_candidates):
        if el in words:
            top_words[el]= my_ngram[el]
           
    top_words= sorted(top_words.items(), key=lambda x: x[1], reverse=True)
    top_words= top_words[:5]
    top_words_and_probabilities = dict(top_words) # This gives the top words as well as the necessary probabilities
    top_words= list(dict(top_words)) 
    
    #return candidates(err_word)
    return top_words, top_words_and_probabilities




## Candidate Ranking

In [11]:
def candidate_ranking (sugg,ngram_result):
    
    dic_sugg= {}
    val= 0
    word= ""
    
    for el in sugg:   
        dic_sugg[el]= ngram_result[el]
    
    for el in dic_sugg:
        if dic_sugg[el]> val:
            val= dic_sugg[el]
            word = el
    
    dic = dict(sorted(dic_sugg.items()))# This is not functioning right
         
    return word

## Spell Checker no update

"""To check for error word, for each word query if it is stored in the dictionary, else, 
get the error word and store its index"""

import re
"""
import all you need here
"""

def spell_checker():
    sentence=  input("Enter text here:")
    sentence= sentence.lower()
    #sent=re.sub(r"[^A-Za-z àáéèọíìịị̀ụụ́ụ̀úùṅṇÄÁÀÉÈỌỌ̀ÒÓỊ̀ỊÌỤÚṄŃḾ’']",'',sentence)#Use regex to clean sentence
    sent=re.sub(r"[^\w\s’]",'',sentence)#Use regex to clean sentence
    
    

    # I would have to turn My_error_sentence to a list
    arr_for_sents = sent.split()
    arr_for_sents_safe_keep= sent.split()

    ### **** dictionary_words_for_edit= "add file for corpus here for edit distance"
    #print(arr_for_sents)


    # Here I initialize the index so that I can store the position of the number so as to change it when the correction is generated 
    # I also keep a list for possible candidates, so I can keep lists for certain indexes in it. I could make this a dictionary, with the index being the key
    arr_index= 0
    poss_candidates= {}
    


    for word in arr_for_sents:

        if t.search(word) == False:


            arr_index= arr_for_sents.index(word) #store the array index   
            
            ngram= ngram_multi_sugg(sent,arr_index) #get ngram suggestions
            
            candidates,candidates_and_prob= candidate_suggestions(ngram,word) #Get the suggestions
            
            correct_word= candidate_ranking(candidates,ngram) #Get the correct word
            

            #add suggestions to dictionary
            poss_candidates[arr_index]= candidates#result[1]

            #replace most-likely word with word in index 
            arr_for_sents[arr_index]= correct_word#result[0]

            #return array index to zero 
            arr_index = 0 



    #print(poss_candidates)
    #print(arr_index)


    #print(f"Corrected Text: {' '.join(arr_for_sents_safe_keep)}")# Prints error sentence
    print(f"Corrected Text: {' '.join(arr_for_sents)}")# prints updated sentence

    
    
    
    
    
    
    
    
    
    
    print(poss_candidates)


    #print out error word and candidate suggestions
    for el in poss_candidates:
        print(f"\nError word:'{arr_for_sents_safe_keep[el]}'\n->suggestions: {poss_candidates[el]}\nCorrection: '{arr_for_sents[el]}'")

## Spell Checker with Uploading sentence

1. This version updates any error word it cannot find back into the sentence.
2. It also updates the sentence as the correction is going on

In [12]:
"""To check for error word, for each word query if it is stored in the dictionary, else, 
get the error word and store its index"""

import re
from colorama import Fore, Back, Style
#print(Fore.RED + 'some red text')
"""
import all you need here
"""

def spell_checker():
    sentence=  input("Enter text here:")
    main_sentence= sentence
    
    sentence= sentence.lower()
    #sent=re.sub(r"[^A-Za-z àáéèọíìịị̀ụụ́ụ̀úùṅṇÄÁÀÉÈỌỌ̀ÒÓỊ̀ỊÌỤÚṄŃḾ’']",'',sentence)#Use regex to clean sentence
    sent=re.sub(r"[^\w\s’]",'',sentence)#Use regex to clean sentence
    
    
    

    # I would have to turn My_error_sentence to a list
    arr_for_sents = sent.split()
    arr_for_sents_safe_keep= sent.split()

    ### **** dictionary_words_for_edit= "add file for corpus here for edit distance"
    #print(arr_for_sents)


    # Here I initialize the index so that I can store the position of the number so as to change it when the correction is generated 
    # I also keep a list for possible candidates, so I can keep lists for certain indexes in it. I could make this a dictionary, with the index being the key
    arr_index= 0
    poss_candidates= {}
    


    for word in arr_for_sents:

        if t.search(word) == False:


            arr_index= arr_for_sents.index(word) #store the array index   
            
            ngram= ngram_multi_sugg(sent,arr_index) #get ngram suggestions
            
            candidates,candidates_and_prob= candidate_suggestions(ngram,word) #Get the suggestions
            
            correct_word= candidate_ranking(candidates,ngram) #Get the correct word
            

            #add suggestions to dictionary
            poss_candidates[arr_index]= candidates  #result[1]
            
            if candidates == []:
                arr_for_sents[arr_index]= (Fore.RED + word+Style.RESET_ALL) #check how to return a colored word back
                
            else: 
                #replace most-likely word with word in index 
                arr_for_sents[arr_index]= (Fore.BLUE + correct_word +Style.RESET_ALL)#correct_word #result[0]

            #return array index to zero 
            arr_index = 0 
            
        #update sent
        sent=' '.join(arr_for_sents)



    #print(poss_candidates)
    #print(arr_index)


    #print(f"Corrected Text: {' '.join(arr_for_sents_safe_keep)}")# Prints error sentence
    
    print(f"\nCorrected Text: {' '.join(arr_for_sents)}")# prints updated sentence

    
    
    
    
    
    
    
    
    
    
    #print(poss_candidates)


    #print out error word and candidate suggestions
    for el in poss_candidates:
        print(f"\n \nError word:'{arr_for_sents_safe_keep[el]}'\n->suggestions: {poss_candidates[el]}\nCorrection: '{arr_for_sents[el]}'")

In [13]:
spell_checker()

Enter text here:hk

Corrected Text: hk

 
Error word:'hk'
->suggestions: []
Correction: 'hk'


# Evaluation Getting predicted_data

In [14]:
import re
from colorama import Fore, Back, Style
#print(Fore.RED + 'some red text')
"""
import all you need here
"""

def spell_checker_1(sentence):
    main_sentence= sentence
    
    sentence= sentence.lower()
    #sent=re.sub(r"[^A-Za-z àáéèọíìịị̀ụụ́ụ̀úùṅṇÄÁÀÉÈỌỌ̀ÒÓỊ̀ỊÌỤÚṄŃḾ’']",'',sentence)#Use regex to clean sentence
    sent=re.sub(r"[^\w\s’]",'',sentence)#Use regex to clean sentence
    
    
    

    # I would have to turn My_error_sentence to a list
    arr_for_sents = sent.split()
    arr_for_sents_safe_keep= sent.split()

    ### **** dictionary_words_for_edit= "add file for corpus here for edit distance"
    #print(arr_for_sents)


    # Here I initialize the index so that I can store the position of the number so as to change it when the correction is generated 
    # I also keep a list for possible candidates, so I can keep lists for certain indexes in it. I could make this a dictionary, with the index being the key
    arr_index= 0
    poss_candidates= {}
    


    for word in arr_for_sents:

        if t.search(word) == False:


            arr_index= arr_for_sents.index(word) #store the array index   
            
            ngram= ngram_multi_sugg(sent,arr_index) #get ngram suggestions
            
            candidates,candidates_and_prob= candidate_suggestions(ngram,word) #Get the suggestions
            
            correct_word= candidate_ranking(candidates,ngram) #Get the correct word
            

            #add suggestions to dictionary
            poss_candidates[arr_index]= candidates  #result[1]
            
            if candidates == []:
                arr_for_sents[arr_index]= (Fore.RED + word+Style.RESET_ALL) #check how to return a colored word back
                
            else: 
                #replace most-likely word with word in index 
                arr_for_sents[arr_index]= (Fore.BLUE + correct_word +Style.RESET_ALL)#correct_word #result[0]

            #return array index to zero 
            arr_index = 0 
            
            try:
                # block raising an exception
                "IndexError: list index out of range"
            except:
                pass # doing nothing on exception
            
        #update sent
        sent=' '.join(arr_for_sents)



    #print(poss_candidates)
    #print(arr_index)


    #print(f"Corrected Text: {' '.join(arr_for_sents_safe_keep)}")# Prints error sentence
    
    
    #print(f"\nCorrected Text: {' '.join(arr_for_sents)}")# prints updated sentence
    

    
    
    
    
    
    
    
    
    
    
    #print(poss_candidates)
    #error_words = ['ie', 'nị']
    #suggestions = [['ihe', 'ike', 'ime', 'iwe', 'e'], ['ndị', 'na', 'dị', 'gị', 'sị']]

    #output = list(zip(error_words, suggestions))

   
    
    err=[]
    sugg=[]
    
    for el in poss_candidates:
        err.append(arr_for_sents_safe_keep[el])
        sugg.append(poss_candidates[el])
        
    output= list(zip(err, sugg))

    #print(output)
    return output
        
    
    


    #print out error word and candidate suggestions
    #for el in poss_candidates:
       # print(f"\n \nError word:'{arr_for_sents_safe_keep[el]}'\n->suggestions: {poss_candidates[el]}\nCorrection: '{arr_for_sents[el]}'")

In [15]:
sent= open("text_file_modified.txt",encoding="utf8")
#sent= open("test.ig.with.errors.txt",encoding="utf8")
#sent= open("testing_my_test_set_err.txt",encoding="utf8")



sent= sent.read()
#sent= sent[:1000]

def checker(x):

    x= x.split("\n")
    my_arr=[]
    
    for el in x: 
        x= spell_checker_1(el)
        
        """This code was put here so that the function
        will not include words with no suggestions
        if len(x)>0:
            combined_data = (el, x)
            my_arr.append(combined_data)
            """
        combined_data = (el, x)
        my_arr.append(combined_data)
        
        
    return my_arr
  

#Remove el with one el in the array



In [16]:
predicted_data=  checker(sent)
#predicted_data= predicted_data [:5]
#predicted_data




## Evaluation

In [17]:
file1 = open("test.ig.txt",encoding="utf8")
#file1 = open("testing_my_test_set_corr.txt",encoding="utf8")
file1= file1.read()
file1= file1.split("\n")
file1_sample= file1[:10]
file1_sample

#file2 = open("test.ig.with.errors.txt",encoding="utf8")# This is supposed to be the sentences the ai corrects
file2 = open("text_file_modified.txt",encoding="utf8")# This is supposed to be the sentences the ai corrects
file2= file2.read()
file2= file2.split("\n")
file2_sample= file2[:10]
file2_sample
#tuple(file2_sample)

#initial version without removing single-word sentences
"""
def gold_data(x,y):
    
    ex=[]
    
    for i in range(len(x)):
        
        z= x[i]+","+ y[i]
        z=z.split(",")
        z=tuple(z)
        ex.append(z)
        
    return ex
"""

def gold_data(x,y):
    
    ex=[]
    
    for i in range(len(x)):
        if len(x[i])>1 and len(y[i])>1:
            z= x[i]+","+ y[i]
            z=z.split(",")
            z=tuple(z)
            ex.append(z)
        
    return ex

#gold_data = gold_data(file2,file1)
#gold_data= gold_data[:5]
gold_data= file1
"""
gold_data=[('iji ubube eme ihe ì na eme nnyocha ndị a iji zere mmerụ ahụ ',
  'iji ubube eme ihe ì na eme nnyocha ndị a iji zere mmerụ ahụ'),
 ('site naka onye nta akụkọ teta ', 'site naka onye nta akụkr teta'),
 ('ọ dị paul mkpa ịgbanwe bọlb ọkụ eletrik dị nihu ụlọ ya ',
  'ọ dị paul mkpa ịgbanwe bọlb ọkụ eletrik dị nihu ụlọ ya'),
 ('ọ dịkwa ya mkpa ihicha windo ndị dị nụlọ elu site nèzí nwunye ya ekwuwo nke a ọtụtụ ugboro ',
  'ọ dịkwa ya mkpa ihicha winda ndị dị nụọ elu site nèzí nwunye ya ekwuwo nke a ọtụtụ ugboro')]"""

#len(file1),len(file2)

"\ngold_data=[('iji ubube eme ihe ì na eme nnyocha ndị a iji zere mmerụ ahụ ',\n  'iji ubube eme ihe ì na eme nnyocha ndị a iji zere mmerụ ahụ'),\n ('site naka onye nta akụkọ teta ', 'site naka onye nta akụkr teta'),\n ('ọ dị paul mkpa ịgbanwe bọlb ọkụ eletrik dị nihu ụlọ ya ',\n  'ọ dị paul mkpa ịgbanwe bọlb ọkụ eletrik dị nihu ụlọ ya'),\n ('ọ dịkwa ya mkpa ihicha windo ndị dị nụlọ elu site nèzí nwunye ya ekwuwo nke a ọtụtụ ugboro ',\n  'ọ dịkwa ya mkpa ihicha winda ndị dị nụọ elu site nèzí nwunye ya ekwuwo nke a ọtụtụ ugboro')]"

## Error Correction 

In [44]:
# ERROR CORRECTION Tp-Fn
# tp: where the model correctly corrects an error_word will be a situation where the correct word is in the list of suggestions
# fp: where a incorrect word is incorrected by the model (word in incorrect but correction not in sugg)
# fn: where an correct word is corrected by the model (word in correct? but not in identified_error)
# tn: where a correct word is ignored (Word in correct and not in identified_errors)


tp=0
fn=0
fp=0
tn=0


#first I get the correct and incorrect words by default sentence
# Then I get the predicted words
# for the words in incorrect, if word in pred_err and suggestion is true elsefn, then tp (this means error is correct)
# for words in incorrect, if word not in pred_errors(i.e incorrect word not flagged), then fn (this means error is correct and no sugg)
# for worsd in correct, if word in pred_errors(a correct word is flagged)then fp, (this means there is no error, error is incorrect ie unavailable)
# for words in correct, if word not in pred_errors(correct word not flagged) then tn

gold_sentence=[]
pred_sentence=[]
pred_error_words=[]

for (gold_sent), (pred_sent, pred_errors) in zip(gold_data, predicted_data):
    gold_sentence.append(gold_sent)
    pred_sentence.append(pred_sent)
    pred_error_words.append(pred_errors)

tp=0
tn=0
fn=0
fp=0
correct_words=[]
incorrect_words=[]
identified_error_words=[]
suggestions=[]

#print(pred_sentence)

for el in range(0,len(gold_sentence)): 
    
    #get the correct and incorrect words for each sentence
    for word in pred_sentence[el].split():
        #print(word)     
        if word in gold_sentence[el]:
            correct_words.append(word)
            
        else:
            incorrect_words.append(word)
    #print(correct_words)
    #print(incorrect_words)
    
    for arr in pred_error_words[el]:
        identified_error_words.append(arr[0])
        suggestions.append(arr[1])
    #print(identified_error_words) 
    #print(suggestions)  
    
    
    # for the words in incorrect, if word in pred_err and suggestion is true elsefn, then tp (this means error is correct)
    # for words in incorrect, if word not in pred_errors(i.e incorrect word not flagged), then fn (this means error is correct and no sugg)
    # for worsd in correct, if word in pred_errors(a correct word is flagged)then fp, (this means there is no error, error is incorrect ie unavailable)
    # for words in correct, if word not in pred_errors(correct word not flagged) then tn
    
    for word in incorrect_words:
        
        #print(word)
        if word in identified_error_words:
            #print(word)
            x= gold_sentence[el].split()
            indx= pred_sentence[el].split().index(word)
            indx2= identified_error_words.index(word)
            #print(suggestions[indx2])
            
            if x[indx] in suggestions[indx2]:
                tp+=1
                #if any(suggestion in gold_words for suggestion in suggestions):
            else:
                fn+=1           
        else:
            fn+=1
            
        
    for word in correct_words:
        
        if word in identified_error_words:
            fp+=1
        else:
            tn+=1
    
    
    
    correct_words=[]
    incorrect_words=[]
    identified_error_words=[]
    suggestions=[]
    
    
"""
#Initial result for 5-words suggestions
recall: 0.8512874463564019
precision: 0.6482874050100741
accuracy: 0.9123453190073499
fi_score: 0.7360471923267439
tp= 81727
tn= 528372
fp= 44339
fn= 14277
""" 
    


#recall = tp / (tp + fn) if (tp + fn) != 0 else 0
#precision = tp / (tp + fp) if (tp + fp) != 0 else 0
precision = tn / (tn + fn) if (tn + fn) != 0 else 0
recall = tn / (tn + fp) if (tn + fp) != 0 else 0 
accuracy= (tp+tn)/(tp + fp + fn + tn) if (tp + fp + fn + tn)!= 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0


print(f'recall: {recall}\nprecision: {precision}\naccuracy: {accuracy}\nfi_score: {f1_score}')
print(f'tp= {tp}\ntn= {tn}\nfp= {fp}\nfn= {fn}')




recall: 0.9861553030635785
precision: 0.9719577131873555
accuracy: 0.9637921802900957
fi_score: 0.9790050372561605
tp= 79519
tn= 561291
fp= 7880
fn= 16194


## Error Detection

In [33]:
#ERROR DETECTION

gold_sentence=[]
pred_sentence=[]
pred_error_words=[]
for (gold_sent), (pred_sent, pred_errors) in zip(gold_data, predicted_data):
    gold_sentence.append(gold_sent)
    pred_sentence.append(pred_sent)
    pred_error_words.append(pred_errors)


tp_d=0
tn_d=0
fn_d=0
fp_d=0
correct_words=[]
incorrect_words=[]
identified_error_words=[]
corr= 0
incorr= 0
iden= 0



for el in range(0,len(gold_sentence)): 
    
    #get the correct and incorrect words for each sentence
    for word in pred_sentence[el].split():
        #print(word)     
        if word in gold_sentence[el]:
        #if t.search(word):
            correct_words.append(word)
            corr+=1
        else:
            incorrect_words.append(word)
            incorr+=1
    #print(correct_words)
    #print(incorrect_words)
    
    for arr in pred_error_words[el]:
        identified_error_words.append(arr[0])
        iden+=1
    #print(identified_error_words)
    #All operations to include the tp-fn are to be here
    
    # ERROR DETECTION Tp-Fn

    #tp and fn: Check if any of the correct words are among the error words. if valid word is err:add to fn else tp
    for word in correct_words:
        if word in identified_error_words:
            #fn.append(word)
            fn_d+=1
            
        else:
            #tp.append(word)
            tp_d+=1
   

    #tn and fp: Check if any of the error words are missing among the error words, add to fp, else tn
    for word in incorrect_words:
        if word in identified_error_words:
            #tn.append(word)
            tn_d+=1
        else:
            #fp.append(word)
            fp_d+=1
        
        
    correct_words=[] #This prevents the whole words from all senteces being uploaded together
    incorrect_words=[]
    identified_error_words=[]


 #ERROR CCORRECTION

"""
#Get the index of incorrect words
incorrect_words=['quck', 'fx', 'jmps', 'ovr', 'dg']

indx = 0  

for word in pred_sentence[0].split():
    #print(word)
    if word in incorrect_words:
        indx= pred_sentence[0].split().index(word)
        print(indx)
"""
        

       
print(f"corr:{corr}\nincorr:{incorr}\niden:{iden}")

corr:569171
incorr:95713
iden:96250


In [43]:
# ERROR DETECTION EVALUATION

recall = tn_d / (tn_d + fp_d) if (tn_d + fp_d) != 0 else 0 
#recall= tp_d/(tp_d + fn_d) if (tp_d + fn_d) != 0 else 0
#precision = tp_d / (tp_d + fp_d) if (tp_d + fp_d) != 0 else 0
precision = tn_d / (tn_d + fn_d) if (tn_d + fn_d) != 0 else 0
accuracy= (tp_d+tn_d)/(tp_d + fp_d + fn_d + tn_d) if (tp_d + fp_d + fn_d + tn_d)!= 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0


print(f'recall: {recall}\nprecision: {precision}\nfi_score: {f1_score}\naccuracy:{accuracy}')
print(f"tp:{tp_d}\ntn:{tn_d}\nfn:{fn_d}\nfp:{fp_d}")


recall: 0.9232810589993
precision: 0.9181298701298701
fi_score: 0.920698259560436
accuracy:0.9771042768362601
tp:561291
tn:88370
fn:7880
fp:7343


#### Suggestion Adequacy

In [45]:

total_suggestions=0
suggestion_score=0
for (gold_sentence), (pred_sentence, pred_error_words) in zip(gold_data, predicted_data):
     #ERROR CCORRECTION

    gold_words = gold_sentence.split()
    pred_words = pred_sentence.split()
    indx= []
    suggestion=[]
    #total_suggestions=[]
    
    for error_word, suggestions in pred_error_words:
        #print(error_word)
        if error_word in pred_words:
            indx.append(pred_words.index(error_word))
            suggestion.append(suggestions)
            #total_suggestions.append(suggestions)#change to +=1
            total_suggestions+=1
    #print(indx)
    #print(suggestion)
    """
    -	Correct suggestion is in the first 3 suggestions = 1 
    -	Correct suggestion is in the last 7 suggestions = 0.5 
    -	Correct suggestion is not in any of the 5 suggestions = -0.5 
    -	No suggestions are returned= 0
    """

    for i in range(len(indx)):
        #print(indx[i])
        if len(suggestion[i]) == 0:
            suggestion_score += 0
        
        elif gold_words[indx[i]] in suggestion[i][:2]:
            #print(gold_words[indx[i]])
            suggestion_score+=1
        
        elif gold_words[indx[i]] in suggestion[i][2:]:
            suggestion_score += 0.5
            
        elif gold_words[indx[i]] not in suggestion[i]:#remeber 0.5 gets subtracted and also no score is added hence its more like a -1.5
            suggestion_score -= 0.5
            
        else:
            #suggestion_score += 0
            pass
        
        """#This code is the code to be reciprocated when adding any new conditions
        elif gold_words[indx[i]] in suggestion[i][:1]:
            suggestion_score+=1
        """ 
        
            
            
suggestion_adequacy= suggestion_score/total_suggestions
    
    
    
#print(suggestion_score)
#print(total_suggestions)
print(suggestion_adequacy)


0.8464831168831168


In [20]:
#ERROR DETECTION

gold_sentence=[]
pred_sentence=[]
pred_error_words=[]
for (gold_sent), (pred_sent, pred_errors) in zip(gold_data, predicted_data):
    gold_sentence.append(gold_sent)
    pred_sentence.append(pred_sent)
    pred_error_words.append(pred_errors)
    

correct_words=[]
incorrect_words=[]
identified_error_words=[]
all_words=[]

for el in range(0,len(gold_sentence)): 
    err_sent=pred_sentence[el].split()
    corr_sent= gold_sentence[el].split()
    
    #get the correct and incorrect words for each sentence
    for word in pred_sentence[el].split():
    #for i in range(0,len(corr_sent)):
        #print(err_sent[i])  
        all_words.append(word)
        
        if word in gold_sentence[el]:
        #if t.search(word):
            correct_words.append(word)
            
        else:
            incorrect_words.append(word)
    #print(correct_words)
    #print(incorrect_words)
    
    for arr in pred_error_words[el]:
        identified_error_words.append(arr[0])
        

        




#print (f"corr:{len(correct_words)}\nincorr:{len(incorrect_words)}\n{len(all_words)}\nident:{len(identified_error_words)}")
#print(incorrect_words)
#print(len(pred_sentence))
#print(len(gold_sentence))
#print(gold_sentence)
#print(pred_sentence)
#print(len(gold_data))
#print(len(predicted_data))



In [163]:
word="mgbe e kwusịrị okwu ndị bú nọgidenụ na anwapụta ihe unu onwe unu bụ nakwa kwe ka okwu chineke na eduzi nzọụkwụ gị kwa ụbọchị nnọkọ nke ụtụtụ g eji okwu bú isi nke isiokwu ya bụ soro chineke na eje ije noge ọgba aghara mechie iji dezie ihe ọ bụla na ekwekọghị ekwekọ netiti vulgate na asụsụ ndị mbụ e ji dee bible nebrija gbara cisneros ume sị mụnyeghachi ọkụ abụọ nke okpukpe anyị asụsụ hibru na nke grik m ga achọta akwụkwọ e ji amụ yoruba ọ ma na e nweghị ihe a ga eji tụnyere aka ọ na enyere jizọs na ume ọ na agba ya gịnị na gịnị mere ka ụjọ tụọ mozis nwaanyị ahụ kwuru sị anyị na arịọ ka a gbatara anyị ọsọ enyemaka ọ dịkwa anyị oké mkpa b olee otú ndị izrel ga esi abụrụ jehova ndị dị nsọ ma mgbe obere oge gafere ha kwụsịrị ịṅụrị ọṅụ nihi n ihe ka agha ha njọ bidoziri ime na britain ọ bu ezie na a na eleda ha anya nihi ụzọ ndụ ha a gbaghaara ndị dị otú ahụ obi ha ziri ezi mmehie ha nihi okwukwe ha nwere nebe ọkpara chineke nọ mmetụta setan na emekwa ka ihe na akawanye nanị njọ ndị ọzọ na ebu ihe na adịghị mma nobi ekpe ekpere ma ọ dị njikere is nụzọ ọ bụla ruru ya aka mee ka anyị kwụsị irubere chineke isi otu nime ha bụ mmadụ na onye na abụghị di ya ma ọ bụ nwunye ya inwe mmekọahụ nụdị ya ọ bịa ya mere mgbe m ghọsịrị dibịa bekee ahọọrọ m ịga nihu na nkà mmụta ọgwụ site nịmụ banyere ọrịa ịmụ banyere ihe ndị e ji mara ọrịa ihe ndị na akpata ya na mmetụta ndị ọ na enwe ọ bụghị ihe ijuanya na ahụ ọkụ ji kwok kit unu ejiriwo ọ na anara ihe ì na echeta a na anụkarị ka ndị mmadụ na ewu ụdị ihe ndị a mgbe ụkọ ego dị o nweghị ihe na ere obi ụtọ karịa ịhụ ka ndị ị na amụrụ baịbụl gbanwee otú ha si ebi ndụ nihi n ha amatala jehova ka ọ dị ugbu a vatican bụ òtù na ekiri ihe a na eme mgbe nile ma ọ bụ mba na esoghị ná mba ndị so nòtù un nnapụta ngosipụta nke obiọma sitere nịhụnanya onye na eji ọmịiko ege ntị na eme ka ahú ruo ndị ọzọ ala kwụsie ike ná mkpebi gị gịnị mechara mee ma ndị ndú okpukpe ndị juu enweghị iche ha ji ha kpọrọ ò pụtara na ihe ọ chọburu ime adịghị mma mbanụ olee otú nwunye bú onye kraịst kwesịrị isi na emeso di ya na ekweghị ekwe otu ihe bụ na nke a na enye ohere iji kpebie kpam kpam ihe iseokwu ndị dị oké mkpa ndị nnupụisi setan kpalitere nogige iden olee ihe ị mụtara nekpere otu onye livaị kpere mgbe ọ na enweghị ike ịga nụlọ nọ chineke ọ dịghị ma ọlị ụkpụrụ ndị b isi nke bible bụ ndị nwere kpọmkwem ihe jikọrọ ha na mmekọrịta anyị na jehova a gwara anyị ka anyị gakwuru jenny bú agadi nwaanyị isi chara awọ bú onye ndị obodo ahụ olee uru ọ bara isi ihe ndị mejupụtara ọgwụ ụfọdụ a na agbanye nahú na abụwanye mmiri ọgwụ ndị a na esighị nọbara nweta o nwere ike ịbụ na ọ na eme gị ihere nihi y oké ìgwè mmadụ ga anọ ya ịnabata ndị e si nọnụ kpọlite mbanụ"
word= word.split()
len(word)

585

# After Testing
- The single quotes in some words have to go
-

In [ ]:
x= 'olee otú ị pụrụ isi mee ka ahú ike gị ka mma site n’ihe oriri ka mma' # Testing the whole of this sentence gives a list index out of range error. This is becasue of the quotes and the space. It was the regex
y= 'n’ezie ọchịchị ọma nke solomọn wetaara ndị ọ na achị udo mee ka ha nọrọ ná nchebe meekwa ka ha nwee obi ụtọ'
x.strip()
print(x)
q= 'olee otú ị pụrụ isi mee ka ahú ike gị ka mma site n’ihx orirx za mna'

# I also need to write a code that performes the whole process for one word at a time

#olee otú ị pụrụ isi mee ka ahú ike gị ka mma site n’ihe oriri ka mma 
# Old test set data with words of 1 occurence
x= "Iguzo mwuli akọnauche nke ndị mmadụ site n'ịchọ mgbanwe abụghị ịda iwu"# I need to prevent my spell checker from removing the ('), I also need to remove words or sentences with low no of occurence (akọnauche)'
x2= "ya mere, mgbe i chere na ị na-agba nkịtị bụ okwu ngosi ogogo, ekwenyeghị m"
x3= "iji wezụga obi abụọ, ebea bụ ihe ole na ole n'ime ndị ọzọ nwoke ma ọbụ nwaanyị ọbụla chọrọ ịzụahịa n'ala abụghị ebe a mụrụ ya:" # ịzụahịa, 

In [ ]:
t.search("ruo")

# 1. I need to correct the trie, it accepts any single letter of the alphabet as a correct word, I need to find the correct 
# single alphabets in Igbo and correct for the. This may not be necessary as the spell checker need not disqualiify any letter 
# as incorrect. I could identify the ones non-native to igbo and remove them from the trie e.g c,q,x

## Questions

1. Does the correction have to make sense in the language or just make sense in context. I.e we correct an error from a sentence and it doesnt give us the word we are expecting from the edit, rather it gives another different word from the word we intended to correct to, but this word is still correct in the context.
2. Do I need to return the text back to the specified format, i.e with the special characters and numbers etc

## Notes
1. Norvig performs much better when used within the application as many sugestions are returned
2. In the original dataset, the special characters set themselves apart with spaces, this makes it easy to use regex in processing them. This is important as a feauture of any dataset that we will be using for the work to train and test the model.